<font size="+3">Time-series Generative Adversarial Network (TimeGAN)</font>

# Imports & Settings

Adapted from the excellent paper by Jinsung Yoon, Daniel Jarrett, and Mihaela van der Schaar:  
[Time-series Generative Adversarial Networks](https://papers.nips.cc/paper/8789-time-series-generative-adversarial-networks),  
Neural Information Processing Systems (NeurIPS), 2019.

- Last updated Date: April 24th 2020
- [Original code](https://bitbucket.org/mvdschaar/mlforhealthlabpub/src/master/alg/timegan/) author: Jinsung Yoon (jsyoon0823@gmail.com)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from pathlib import Path
from tqdm import tqdm

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, RNN, GRUCell, Input
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using CPU


In [4]:
sns.set_style('white')

# Experiment Path

In [5]:
results_path = Path('time_gan')
if not results_path.exists():
    results_path.mkdir()

In [6]:
experiment = 0

In [7]:
log_dir = results_path / f'experiment_{experiment:02}'
if not log_dir.exists():
    log_dir.mkdir(parents=True)

In [8]:
hdf_store = results_path / 'TimeSeriesGAN.h5'

# Prepare Data

## Parameters

In [9]:
#seq_len = 24
n_seq = 1
batch_size = 1

In [10]:
tickers = ['BRK-B']

In [11]:
def select_data():
    df = pd.read_csv('BRK-B.csv')

In [12]:
select_data()

In [13]:
df = pd.read_csv('BRK-B.csv', index_col = 0)
# df = pd.concat([df['AdjClose'], axis=1)

In [14]:
df_new = df.copy()

In [15]:
#train test split

In [16]:
train_number = int(len(df_new) * 0.8)

In [17]:
#tscv = TimeSeriesSplit(n_splits = 2, test_size = test_number)

In [18]:
#for train_index, test_index in tscv.split(X):
    #X_train, X_test = X[train_index], X[test_index]

train, test = df_new[0:train_number], df_new[train_number:]
seq_len = len(test)

In [65]:
len(test)

252

## Plot Series

In [19]:
# axes = train.div(train.iloc[0]).plot(subplots=True,
#                                figsize=(14, 6),
#                                layout=(3, 2),
#                                title=tickers,
#                                legend=False,
#                                rot=0,
#                                lw=1, 
#                                color='k')
# for ax in axes.flatten():
#     ax.set_xlabel('')

# plt.suptitle('Normalized Price Series')
# plt.gcf().tight_layout()
# sns.despine();

## Correlation

In [20]:
# sns.clustermap(train.corr(),
#                annot=True,
#                fmt='.2f',
#                cmap=sns.diverging_palette(h_neg=20,
#                                           h_pos=220), center=0);

## Normalize Data

In [21]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(train[['AdjClose']]).astype(np.float32)

## Create rolling window sequences

In [22]:
data = []
for i in range(len(train) - seq_len):
    data.append(scaled_data[i:i + seq_len])

n_windows = len(data)

## Create tf.data.Dataset

In [23]:
real_series = (tf.data.Dataset
               .from_tensor_slices(data)
               .shuffle(buffer_size=n_windows)
               .batch(batch_size))
real_series_iter = iter(real_series.repeat())

## Set up random series generator

In [24]:
def make_random_data():
    while True:
        yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

We use the Python generator to feed a `tf.data.Dataset` that continues to call the random number generator as long as necessary and produces the desired batch size.

In [25]:
random_series = iter(tf.data.Dataset
                     .from_generator(make_random_data, output_types=tf.float32)
                     .batch(batch_size)
                     .repeat())

In [26]:
#next(random_series)

# TimeGAN Components

The design of the TimeGAN components follows the author's sample code.

##  Network Parameters

In [27]:
hidden_dim = 24
num_layers = 3

## Set up logger

In [28]:
writer = tf.summary.create_file_writer(log_dir.as_posix())

## Input place holders

In [29]:
X = Input(shape=[seq_len, n_seq], name='RealData')
Z = Input(shape=[seq_len, n_seq], name='RandomData')

## RNN block generator

We keep it very simple and use a very similar architecture for all four components. For a real-world application, they should be tailored to the data.

In [30]:
def make_rnn(n_layers, hidden_units, output_units, name):
    return Sequential([GRU(units=hidden_units,
                           return_sequences=True,
                           name=f'GRU_{i + 1}') for i in range(n_layers)] +
                      [Dense(units=output_units,
                             activation='sigmoid',
                             name='OUT')], name=name)

## Embedder & Recovery

In [31]:
embedder = make_rnn(n_layers=3, 
                    hidden_units=hidden_dim, 
                    output_units=hidden_dim, 
                    name='Embedder')
recovery = make_rnn(n_layers=3, 
                    hidden_units=hidden_dim, 
                    output_units=n_seq, 
                    name='Recovery')

## Generator & Discriminator

In [32]:
generator = make_rnn(n_layers=3, 
                     hidden_units=hidden_dim, 
                     output_units=hidden_dim, 
                     name='Generator')
discriminator = make_rnn(n_layers=3, 
                         hidden_units=hidden_dim, 
                         output_units=1, 
                         name='Discriminator')
supervisor = make_rnn(n_layers=2, 
                      hidden_units=hidden_dim, 
                      output_units=hidden_dim, 
                      name='Supervisor')

# TimeGAN Training

## Settings

In [33]:
train_steps = 10000
gamma = 1

## Generic Loss Functions

In [34]:
mse = MeanSquaredError()
bce = BinaryCrossentropy()

# Phase 1: Autoencoder Training

## Architecture

In [35]:
H = embedder(X)
X_tilde = recovery(H)

autoencoder = Model(inputs=X,
                    outputs=X_tilde,
                    name='Autoencoder')

In [36]:
autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RealData (InputLayer)        [(None, 252, 1)]          0         
_________________________________________________________________
Embedder (Sequential)        (None, 252, 24)           9744      
_________________________________________________________________
Recovery (Sequential)        (None, 252, 1)            10825     
Total params: 20,569
Trainable params: 20,569
Non-trainable params: 0
_________________________________________________________________


In [37]:
plot_model(autoencoder,
           to_file=(results_path / 'autoencoder.png').as_posix(),
           show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Autoencoder Optimizer

In [38]:
autoencoder_optimizer = Adam()

## Autoencoder Training Step

In [39]:
@tf.function
def train_autoencoder_init(x):
    with tf.GradientTape() as tape:
        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss_0 = 10 * tf.sqrt(embedding_loss_t0)

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss_0, var_list)
    autoencoder_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

## Autoencoder Training Loop

In [40]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter)
    step_e_loss_t0 = train_autoencoder_init(X_)
    with writer.as_default():
        tf.summary.scalar('Loss Autoencoder Init', step_e_loss_t0, step=step)

100%|██████████| 10000/10000 [27:49<00:00,  5.99it/s]


## Persist model

In [41]:
autoencoder.save(log_dir / 'autoencoder')

INFO:tensorflow:Assets written to: time_gan/experiment_00/autoencoder/assets


INFO:tensorflow:Assets written to: time_gan/experiment_00/autoencoder/assets


# Phase 2: Supervised training

## Define Optimizer

In [42]:
supervisor_optimizer = Adam()

## Train Step

In [43]:
@tf.function
def train_supervisor(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        g_loss_s = mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

    var_list = supervisor.trainable_variables
    gradients = tape.gradient(g_loss_s, var_list)
    supervisor_optimizer.apply_gradients(zip(gradients, var_list))
    return g_loss_s

## Training Loop

In [44]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter)
    step_g_loss_s = train_supervisor(X_)
    with writer.as_default():
        tf.summary.scalar('Loss Generator Supervised Init', step_g_loss_s, step=step)

100%|██████████| 10000/10000 [13:10<00:00, 12.65it/s]


## Persist Model

In [45]:
# supervisor.save(log_dir / 'supervisor')

# Joint Training

## Generator

### Adversarial Architecture - Supervised

In [46]:
E_hat = generator(Z)
H_hat = supervisor(E_hat)
Y_fake = discriminator(H_hat)

adversarial_supervised = Model(inputs=Z,
                               outputs=Y_fake,
                               name='AdversarialNetSupervised')

In [47]:
adversarial_supervised.summary()

Model: "AdversarialNetSupervised"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RandomData (InputLayer)      [(None, 252, 1)]          0         
_________________________________________________________________
Generator (Sequential)       (None, 252, 24)           9744      
_________________________________________________________________
Supervisor (Sequential)      (None, 252, 24)           7800      
_________________________________________________________________
Discriminator (Sequential)   (None, 252, 1)            10825     
Total params: 28,369
Trainable params: 28,369
Non-trainable params: 0
_________________________________________________________________


In [48]:
plot_model(adversarial_supervised, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


### Adversarial Architecture in Latent Space

In [49]:
Y_fake_e = discriminator(E_hat)

adversarial_emb = Model(inputs=Z,
                    outputs=Y_fake_e,
                    name='AdversarialNet')

In [50]:
adversarial_emb.summary()

Model: "AdversarialNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RandomData (InputLayer)      [(None, 252, 1)]          0         
_________________________________________________________________
Generator (Sequential)       (None, 252, 24)           9744      
_________________________________________________________________
Discriminator (Sequential)   (None, 252, 1)            10825     
Total params: 20,569
Trainable params: 20,569
Non-trainable params: 0
_________________________________________________________________


In [51]:
plot_model(adversarial_emb, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


### Mean & Variance Loss

In [52]:
X_hat = recovery(H_hat)
synthetic_data = Model(inputs=Z,
                       outputs=X_hat,
                       name='SyntheticData')

In [53]:
synthetic_data.summary()

Model: "SyntheticData"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RandomData (InputLayer)      [(None, 252, 1)]          0         
_________________________________________________________________
Generator (Sequential)       (None, 252, 24)           9744      
_________________________________________________________________
Supervisor (Sequential)      (None, 252, 24)           7800      
_________________________________________________________________
Recovery (Sequential)        (None, 252, 1)            10825     
Total params: 28,369
Trainable params: 28,369
Non-trainable params: 0
_________________________________________________________________


In [54]:
plot_model(synthetic_data, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [55]:
def get_generator_moment_loss(y_true, y_pred):
    y_true_mean, y_true_var = tf.nn.moments(x=y_true, axes=[0])
    y_pred_mean, y_pred_var = tf.nn.moments(x=y_pred, axes=[0])
    g_loss_mean = tf.reduce_mean(tf.abs(y_true_mean - y_pred_mean))
    g_loss_var = tf.reduce_mean(tf.abs(tf.sqrt(y_true_var + 1e-6) - tf.sqrt(y_pred_var + 1e-6)))
    return g_loss_mean + g_loss_var

## Discriminator

### Architecture: Real Data

In [56]:
Y_real = discriminator(H)
discriminator_model = Model(inputs=X,
                            outputs=Y_real,
                            name='DiscriminatorReal')

In [57]:
discriminator_model.summary()

Model: "DiscriminatorReal"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
RealData (InputLayer)        [(None, 252, 1)]          0         
_________________________________________________________________
Embedder (Sequential)        (None, 252, 24)           9744      
_________________________________________________________________
Discriminator (Sequential)   (None, 252, 1)            10825     
Total params: 20,569
Trainable params: 20,569
Non-trainable params: 0
_________________________________________________________________


In [58]:
plot_model(discriminator_model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Optimizers

In [59]:
generator_optimizer = Adam()
discriminator_optimizer = Adam()
embedding_optimizer = Adam()

## Generator Train Step

In [60]:
@tf.function
def train_generator(x, z):
    with tf.GradientTape() as tape:
        y_fake = adversarial_supervised(z)
        generator_loss_unsupervised = bce(y_true=tf.ones_like(y_fake),
                                          y_pred=y_fake)

        y_fake_e = adversarial_emb(z)
        generator_loss_unsupervised_e = bce(y_true=tf.ones_like(y_fake_e),
                                            y_pred=y_fake_e)
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_hat = synthetic_data(z)
        generator_moment_loss = get_generator_moment_loss(x, x_hat)

        generator_loss = (generator_loss_unsupervised +
                          generator_loss_unsupervised_e +
                          100 * tf.sqrt(generator_loss_supervised) +
                          100 * generator_moment_loss)

    var_list = generator.trainable_variables + supervisor.trainable_variables
    gradients = tape.gradient(generator_loss, var_list)
    generator_optimizer.apply_gradients(zip(gradients, var_list))
    return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss

## Embedding Train Step

In [61]:
@tf.function
def train_embedder(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss = 10 * tf.sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss, var_list)
    embedding_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

## Discriminator Train Step

In [62]:
@tf.function
def get_discriminator_loss(x, z):
    y_real = discriminator_model(x)
    discriminator_loss_real = bce(y_true=tf.ones_like(y_real),
                                  y_pred=y_real)

    y_fake = adversarial_supervised(z)
    discriminator_loss_fake = bce(y_true=tf.zeros_like(y_fake),
                                  y_pred=y_fake)

    y_fake_e = adversarial_emb(z)
    discriminator_loss_fake_e = bce(y_true=tf.zeros_like(y_fake_e),
                                    y_pred=y_fake_e)
    return (discriminator_loss_real +
            discriminator_loss_fake +
            gamma * discriminator_loss_fake_e)

In [63]:
@tf.function
def train_discriminator(x, z):
    with tf.GradientTape() as tape:
        discriminator_loss = get_discriminator_loss(x, z)

    var_list = discriminator.trainable_variables
    gradients = tape.gradient(discriminator_loss, var_list)
    discriminator_optimizer.apply_gradients(zip(gradients, var_list))
    return discriminator_loss

## Training Loop

In [64]:
step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
for step in range(train_steps):
    # Train generator (twice as often as discriminator)
    for kk in range(2):
        X_ = next(real_series_iter)
        Z_ = next(random_series)

        # Train generator
        step_g_loss_u, step_g_loss_s, step_g_loss_v = train_generator(X_, Z_)
        # Train embedder
        step_e_loss_t0 = train_embedder(X_)

    X_ = next(real_series_iter)
    Z_ = next(random_series)
    step_d_loss = get_discriminator_loss(X_, Z_)
    if step_d_loss > 0.15:
        step_d_loss = train_discriminator(X_, Z_)

    if step % 1000 == 0:
        print(f'{step:6,.0f} | d_loss: {step_d_loss:6.4f} | g_loss_u: {step_g_loss_u:6.4f} | '
              f'g_loss_s: {step_g_loss_s:6.4f} | g_loss_v: {step_g_loss_v:6.4f} | e_loss_t0: {step_e_loss_t0:6.4f}')

    with writer.as_default():
        tf.summary.scalar('G Loss S', step_g_loss_s, step=step)
        tf.summary.scalar('G Loss U', step_g_loss_u, step=step)
        tf.summary.scalar('G Loss V', step_g_loss_v, step=step)
        tf.summary.scalar('E Loss T0', step_e_loss_t0, step=step)
        tf.summary.scalar('D Loss', step_d_loss, step=step)

     0 | d_loss: 1.9793 | g_loss_u: 0.8168 | g_loss_s: 0.0008 | g_loss_v: 0.1916 | e_loss_t0: 0.0247
 1,000 | d_loss: 0.1001 | g_loss_u: 5.1893 | g_loss_s: 0.0000 | g_loss_v: 0.1330 | e_loss_t0: 0.0027
 2,000 | d_loss: 0.4159 | g_loss_u: 2.6722 | g_loss_s: 0.0001 | g_loss_v: 0.0922 | e_loss_t0: 0.0029
 3,000 | d_loss: 0.6958 | g_loss_u: 2.4662 | g_loss_s: 0.0001 | g_loss_v: 0.1990 | e_loss_t0: 0.0079
 4,000 | d_loss: 1.9132 | g_loss_u: 1.1776 | g_loss_s: 0.0000 | g_loss_v: 0.2198 | e_loss_t0: 0.0044
 5,000 | d_loss: 1.5433 | g_loss_u: 1.7990 | g_loss_s: 0.0000 | g_loss_v: 0.1573 | e_loss_t0: 0.0039
 6,000 | d_loss: 0.9129 | g_loss_u: 2.5347 | g_loss_s: 0.0000 | g_loss_v: 0.1552 | e_loss_t0: 0.0016
 7,000 | d_loss: 1.0953 | g_loss_u: 2.4826 | g_loss_s: 0.0000 | g_loss_v: 0.0734 | e_loss_t0: 0.0035
 8,000 | d_loss: 0.8632 | g_loss_u: 2.8453 | g_loss_s: 0.0000 | g_loss_v: 0.3107 | e_loss_t0: 0.0026
 9,000 | d_loss: 0.3066 | g_loss_u: 2.2648 | g_loss_s: 0.0001 | g_loss_v: 0.2881 | e_loss_t

## Persist Synthetic Data Generator

In [65]:
synthetic_data.save(log_dir / 'synthetic_data')

INFO:tensorflow:Assets written to: time_gan/experiment_00/synthetic_data/assets


INFO:tensorflow:Assets written to: time_gan/experiment_00/synthetic_data/assets


In [66]:
#train_discriminator(X_, Z_)

# Generate Synthetic Data

In [67]:
generated_data = []
for i in range(int(n_windows / batch_size)):
    Z_ = next(random_series)
    d = synthetic_data(Z_)
    generated_data.append(d)

In [68]:
len(generated_data)

6

In [69]:
generated_data = np.array(np.vstack(generated_data))
generated_data.shape

(768, 24, 6)

In [70]:
np.save(log_dir / 'generated_data.npy', generated_data)

## Rescale

In [71]:
generated_data = (scaler.inverse_transform(generated_data
                                           .reshape(-1, n_seq))
                  .reshape(-1, seq_len, n_seq))
generated_data.shape

(768, 24, 6)

In [ ]:
#fit test set into our model

In [151]:
# data_test = []
# for i in range(len(test) - seq_len):
#     data_test.append(scaled_data[i:i + seq_len])

# n_windows = len(data_test)

In [152]:
data_test = []
for i in range(len(test)):
    data_test.append(scaled_data[i])

n_windows = len(data_test)

In [72]:
# def make_random_data():
#     while True:
#         yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

In [ ]:
# random_series = iter(tf.data.Dataset
#                      .from_generator(make_random_data, output_types=tf.float32)
#                      .batch(batch_size)
#                      .repeat())

In [ ]:
#synthetic_data.save(log_dir / 'synthetic_data_test')

In [171]:
generated_data = []
for i in range(int(n_windows / batch_size)):
    Z_ = next(random_series)
    d = synthetic_data(Z_)
    generated_data.append(d)

In [172]:
len(generated_data)

252

In [183]:
# generated_data = (scaler.inverse_transform(generated_data
#                                            .reshape(-1, n_seq))
#                   .reshape(-1, seq_len, n_seq))
# generated_data.shape
generated_data_re = (scaler.inverse_transform(generated_data[99][0]))
generated_data_re.shape

(252, 1)

In [182]:
generated_data_re

array([[199.11626865],
       [205.0990975 ],
       [205.39366202],
       [202.29052326],
       [203.49533618],
       [203.29241273],
       [204.48661623],
       [206.70578348],
       [208.67263355],
       [208.90247491],
       [206.47152008],
       [203.52945556],
       [202.10574624],
       [202.61585683],
       [204.39232594],
       [207.1111997 ],
       [209.48293013],
       [210.99255139],
       [211.63188546],
       [211.0391707 ],
       [209.96436752],
       [208.46718608],
       [206.37168094],
       [202.69596201],
       [198.51783728],
       [193.83288468],
       [191.61253555],
       [190.53557893],
       [190.06073455],
       [190.20380009],
       [190.63678775],
       [190.46633355],
       [190.4416417 ],
       [190.64011806],
       [190.96023831],
       [191.2904171 ],
       [191.70308989],
       [191.99090365],
       [192.09504965],
       [196.26294809],
       [196.18816389],
       [196.66262265],
       [196.88752614],
       [196

In [169]:
real = pd.DataFrame(test['AdjClose'])
synthetic_daily = pd.DataFrame(generated_data, index = real.index)
real['synthetic_daily'] = synthetic_daily
#result = pd.concat([real, synthetic_daily[0]], ignore_index = True, axis = 1)

In [150]:
real.to_csv('Daily_data.csv')